# Pysam

Pysam è un package che mette a disposizione le funzionalità per manipolare file in formato SAM/BAM.

Importare il modulo `pysam`

In [2]:
import pysam

## Come leggere gli allineamenti da un file `BAM`

`AlignmentFile` è la classe che rappresenta un set di allineamenti.

Un oggetto di tipo `AlignmentFile` può essere costruito a partire da un file SAM/BAM nel seguente modo: 

    samfile = pysam.AlignmentFile(sam_file_name, 'r')
    bamfile = pysam.AlignmentFile(bam_file_name, 'rb')

Importare la classe `AlignmentFile`.

In [3]:
from pysam import AlignmentFile

In [4]:
help(AlignmentFile)

Help on class AlignmentFile in module pysam.libcalignmentfile:

class AlignmentFile(pysam.libchtslib.HTSFile)
 |  AlignmentFile(filepath_or_object, mode=None, template=None,
 |  reference_names=None, reference_lengths=None, text=NULL,
 |  header=None, add_sq_text=False, check_header=True, check_sq=True,
 |  reference_filename=None, filename=None, index_filename=None,
 |  filepath_index=None, require_index=False, duplicate_filehandle=True,
 |  ignore_truncation=False, threads=1)
 |  
 |  A :term:`SAM`/:term:`BAM`/:term:`CRAM` formatted file.
 |  
 |  If `filepath_or_object` is a string, the file is automatically
 |  opened. If `filepath_or_object` is a python File object, the
 |  already opened file will be used.
 |  
 |  If the file is opened for reading and an index exists (if file is BAM, a
 |  .bai file or if CRAM a .crai file), it will be opened automatically.
 |  `index_filename` may be specified explicitly. If the index is not named
 |  in the standard manner, not located in the 

### Costruire l'oggetto `AlignmentFile` a partire dal file `sample.bam`

Indicizzare il file `sample.bam` tramite la funzione `index()`.

In [5]:
pysam.index('./sample.bam')

''

Costruire l'oggetto `AlignmentFile`.

In [6]:
bamfile = AlignmentFile('./sample.bam', 'rb')

Il metodo `get_index_statistics()` fornisce qualche informazione generale.

In [7]:
bamfile.get_index_statistics()

[IndexStats(contig='X', mapped=25008, unmapped=0, total=25008)]

Il numero di reads mappati e non mappati si ottiene accedendo a `mapped` e `unmapped`:

In [8]:
bamfile.mapped

25008

In [9]:
bamfile.unmapped

0

Per ottenere le references coinvolte nel file BAM:

In [10]:
bamfile.references

('X',)

Per ottenere il numero di references coinvolte nel file:

In [11]:
bamfile.nreferences

1

Per ottenere le lunghezze delle references coinvolte nel file:

In [12]:
bamfile.lengths

(23542271,)

Per ottenere la lunghezza di una reference:

In [14]:
bamfile.get_reference_length('X')

23542271

Per ottenere il numero di reads mappati a una determinata regione della reference:

In [15]:
bamfile.count('X', 280000, 300000)

25008

## Come ottenere gli allineamenti dall'oggetto `AlignmentFile`

Le istruzioni: 

    bamfile.fetch()
    bamfile.fetch(ref_name)
    bamfile.fetch(ref_name, start_region, end_region)
    
restituiscono (rispettivamente) un iteratore contenente:
- tutti gli allineamenti del BAM file
- tutti gli allineamenti che si riferiscono alla reference `ref_name`
- tutti gli allineamenti che si riferiscono alla sottoregione tra le posizioni `start_region` e `end_region`della reference `ref_name`.

Un singolo allineamento tra query e reference è un oggetto di tipo `AlignedSegment`.

---
    
Si estraggano tutti gli allineamenti del BAM file.

In [16]:
all_alignments = bamfile.fetch()

Il numero di allineamenti estratti è:

In [17]:
len(list(all_alignments))

25008

Si estraggano ora tutti gli allineamenti relativa alla reference `X`.

In [18]:
all_alignments = bamfile.fetch('X')

In [19]:
len(list(all_alignments))

25008

Si estraggano infine tutti gli allineamenti relativa alla reference `X` che cadono nella regione tra le posizioni `280000` e `285000`.

In [20]:
all_alignments = bamfile.fetch('X', 280000, 285000)

In [21]:
len(list(all_alignments))

3170

L'istruzione:

    bamfile.head(n)

restituisce i primi `n` allineamenti del BAM file.

---

Si estraggano i primi 1000 allineamenti.

In [22]:
first_alignment = bamfile.head(1000)

In [24]:
len(list(first_alignment))

1000

### Come manipolare gli allineamenti

In [25]:
all_alignments = bamfile.fetch()
all_alignments = list(all_alignments)

In [26]:
help(pysam.libcalignedsegment.AlignedSegment)

Help on class AlignedSegment in module pysam.libcalignedsegment:

class AlignedSegment(builtins.object)
 |  AlignedSegment(AlignmentHeader header=None)
 |  Class representing an aligned segment.
 |  
 |      This class stores a handle to the samtools C-structure representing
 |      an aligned read. Member read access is forwarded to the C-structure
 |      and converted into python objects. This implementation should be fast,
 |      as only the data needed is converted.
 |  
 |      For write access, the C-structure is updated in-place. This is
 |      not the most efficient way to build BAM entries, as the variable
 |      length data is concatenated and thus needs to be resized if
 |      a field is updated. Furthermore, the BAM entry might be
 |      in an inconsistent state.
 |  
 |      One issue to look out for is that the sequence should always
 |      be set *before* the quality scores. Setting the sequence will
 |      also erase any quality scores that were set previously.


Recuperare il primo allineamento della lista.

In [27]:
first_alignment = all_alignments[0]

Gli oggetti `AlignedSegment` hanno una serie di attributi che descrivono l'allineamento.

In [28]:
first_alignment.cigarstring

'100M'

In [29]:
first_alignment.cigartuples

[(0, 100)]

In [30]:
first_alignment.flag

0

In [31]:
first_alignment.is_paired

False

In [32]:
first_alignment.is_reverse

False

In [33]:
first_alignment.is_secondary

False

In [34]:
first_alignment.mapping_quality

255

In [35]:
first_alignment.reference_start

283185

In [36]:
first_alignment.query_name

'FBtr0300326_e_2967_X_283185'

In [37]:
first_alignment.query_qualities

array('B', [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [38]:
first_alignment.query_alignment_sequence

'GCCCTTGCATTTATTTCCAGTTTTGGTTTTTTTTTTGTATTTATCAATTACATTTATAATTAATTATACACCATGTATATATATGTATTTATATACTATA'

Gli oggetti `AlignedSegment` hanno una serie di metodi tra i quali:

In [39]:
first_alignment.get_tags()

[('NH', 1), ('HI', 1), ('AS', 98), ('nM', 0)]

In [40]:
first_alignment.get_forward_sequence()

'GCCCTTGCATTTATTTCCAGTTTTGGTTTTTTTTTTGTATTTATCAATTACATTTATAATTAATTATACACCATGTATATATATGTATTTATATACTATA'

In [68]:
#first_alignment.get_reference_sequence()

In [41]:
first_alignment.to_dict()

{'name': 'FBtr0300326_e_2967_X_283185',
 'flag': '0',
 'ref_name': 'X',
 'ref_pos': '283186',
 'map_quality': '255',
 'cigar': '100M',
 'next_ref_name': '*',
 'next_ref_pos': '0',
 'length': '0',
 'seq': 'GCCCTTGCATTTATTTCCAGTTTTGGTTTTTTTTTTGTATTTATCAATTACATTTATAATTAATTATACACCATGTATATATATGTATTTATATACTATA',
 'qual': '####################################################################################################',
 'tags': ['NH:i:1', 'HI:i:1', 'AS:i:98', 'nM:i:0']}

In [43]:
print(first_alignment.to_string())

FBtr0300326_e_2967_X_283185	0	X	283186	255	100M	*	0	0	GCCCTTGCATTTATTTCCAGTTTTGGTTTTTTTTTTGTATTTATCAATTACATTTATAATTAATTATACACCATGTATATATATGTATTTATATACTATA	####################################################################################################	NH:i:1	HI:i:1	AS:i:98	nM:i:0


## Qualche metodo di `AlignmentFile`

- `find_introns()` restituisce in un oggetto `Counter` gli introni e il loro supporto in termini di reads allineati

       bamfile.find_introns(alignment_iterator)

Per trovare tutti gli introni supportati dagli allineamenti del file in input basta scrivere:

In [44]:
bamfile.find_introns(bamfile.fetch())

Counter({(286891, 286948): 1255,
         (287041, 287328): 794,
         (287041, 289040): 468,
         (287526, 289040): 707,
         (289272, 289870): 530,
         (289272, 290252): 730,
         (289943, 290252): 593,
         (290596, 294712): 741,
         (290596, 294822): 586})

- `pileup()`, restituisce un iteratore contenente un oggetto `PileupColumn` per ognuna delle basi coperte dagli allineamenti che cadono nella regione specificata come *argomento*.

       bamfile.pileup()
       bamfile.pileup(ref_name)
       bamfile.pileup(ref_name, start_region, end_region)

Ogni base della reference coperta da allineamenti viene chiamata *pileup column*.

---

Trovare tutte le basi della reference coperte dagli allineamenti presenti nel BAM file.

In [72]:
#help(pysam.libcalignedsegment.PileupColumn)

In [45]:
pileup_iter = bamfile.pileup()

In [46]:
pileup_columns = list(pileup_iter)

Gli allineamenti presenti nel BAM file coprono un totale di:

In [47]:
len(pileup_columns)

11777

Accedere alla 1000-esima colonna di *pileup* (base della reference) e:

- settare la qualità minima a 0

In [49]:
pileup_columns[999].set_min_base_quality(0)

- accedere alla posizione sulla reference

In [50]:
pileup_columns[999].pos

284184

- recuperare il numero di allineamenti (segmenti) che coprono la base.

In [51]:
pileup_columns[999].nsegments

180

In [52]:
pileup_columns[999].get_num_aligned()

180

- ottenere gli identificatori dei reads (queries) che coprono la base.

In [53]:
pileup_columns[999].get_query_names()

['FBtr0342963_e_4202_X_294862',
 'FBtr0342963_e_6192_X_294862',
 'FBtr0342963_e_7396_X_294862',
 'FBtr0070103_e_4896_X_294862',
 'FBtr0342963_e_1514_X_294862',
 'FBtr0342963_e_4365_X_294862',
 'FBtr0070103_e_1858_X_294862',
 'FBtr0070103_e_2397_X_294862',
 'FBtr0070103_e_5063_X_294862',
 'FBtr0300326_e_4543_X_294861',
 'FBtr0070103_e_519_X_294861',
 'FBtr0070103_e_7416_X_294861',
 'FBtr0342963_e_2335_X_294861',
 'FBtr0342963_e_4019_X_294861',
 'FBtr0342963_e_4425_X_294861',
 'FBtr0342963_e_4688_X_294861',
 'FBtr0300326_e_2872_X_294861',
 'FBtr0070103_e_1324_X_294861',
 'FBtr0070103_e_4777_X_294861',
 'FBtr0070103_e_6318_X_294861',
 'FBtr0070103_e_8286_X_294861',
 'FBtr0342963_e_4202_X_294862',
 'FBtr0342963_e_6192_X_294862',
 'FBtr0342963_e_7396_X_294862',
 'FBtr0070103_e_4896_X_294862',
 'FBtr0342963_e_1514_X_294862',
 'FBtr0342963_e_4365_X_294862',
 'FBtr0070103_e_1858_X_294862',
 'FBtr0070103_e_2397_X_294862',
 'FBtr0070103_e_5063_X_294862',
 'FBtr0342963_e_4688_X_294861',
 'FBtr030

- ottenere la lista degli allineamenti che coprono la base.

In [54]:
pileup_columns[999].pileups

Per il primo oggetto `PileupRead`, accedere alla posizione sul read che corrisponde alla base di pileup sulla *reference*.

In [55]:
pileup_columns[999].pileups[0].query_position

99

L'allineamento del primo read "impilato" è:

In [56]:
pileup_columns[999].pileups[0].alignment